In [1]:
import os

USE_GPU = 1

if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES']='0'
else:
    os.environ['CUDA_VISIBLE_DEVICES']=''

import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-06-15 06:41:28.289702: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 06:41:28.289738: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 06:41:28.289768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 06:41:28.296994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  1


2024-06-15 06:41:29.965972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-15 06:41:29.972004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-15 06:41:29.972219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
import pandas as pd
from functools import reduce
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error
import math
from keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt


column_names = [
    'ID', 'PARAM', 'TYPE', 'YEAR', 'DD',
    'Jan', 'Jan_SYM', 'Feb', 'Feb_SYM', 'Mar', 'Mar_SYM',
    'Apr', 'Apr_SYM', 'May', 'May_SYM', 'Jun', 'Jun_SYM',
    'Jul', 'Jul_SYM', 'Aug', 'Aug_SYM', 'Sep', 'Sep_SYM',
    'Oct', 'Oct_SYM', 'Nov', 'Nov_SYM', 'Dec', 'Dec_SYM'
]

MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

data = pd.read_csv('data/credit_hydrometric_data_all_stations.csv', skiprows=2, names=column_names)

In [3]:
_02HB018 = data[data['ID'] == '02HB018'].reset_index(drop=True)

_02HB025 = data[data['ID'] == '02HB025'].reset_index(drop=True)

_02HB031 = data[data['ID'] == '02HB031'].reset_index(drop=True)

_02HB001 = data[data['ID'] == '02HB001'].reset_index(drop=True)

_02HB013 = data[data['ID'] == '02HB013'].reset_index(drop=True)

_02HB029 = data[data['ID'] == '02HB029'].reset_index(drop=True)  ### TARGET


In [4]:
import pandas as pd
import calendar

def is_valid_date(year, month, day):
    try:
        m = calendar.monthrange(year, month)
        if day > m[1]:
            return False
        else:
            return True
    except Exception as e:
        print(e)
        return False


def split_by_param_1_2(df: pd.DataFrame):
    data1 = df[df['PARAM'] == 1].reset_index(drop=True)
    data2 = df[df['PARAM'] == 2].reset_index(drop=True)
    return data1, data2

def wateroffice_to_timeseries(df : pd.DataFrame):
    data_dict = {'timestamp': [], 'value': []}

    for i, row in df.iterrows():
        year = row['YEAR']
        day = row['DD']
        for month in MONTHS :
            if is_valid_date(year, MONTHS.index(month) + 1, day):

                timestamp = pd.Timestamp(f'{year}-{month}-{day}')
                val = df.at[i, f'{month}']
                data_dict['timestamp'].append(timestamp)
                data_dict['value'].append(val)

    new_data = pd.DataFrame(data_dict)
    return new_data


def merge_param_1_2(df_left : pd.DataFrame, df_right : pd.DataFrame):
    return pd.merge(df_left, df_right, on='timestamp', how='inner')



In [5]:
_02HB018_1, _02HB018_2 = split_by_param_1_2(_02HB018)
_02HB025_1, _02HB025_2 = split_by_param_1_2(_02HB025)
_02HB031_1, _02HB031_2 = split_by_param_1_2(_02HB031)
_02HB001_1, _02HB001_2 = split_by_param_1_2(_02HB001)
_02HB013_1, _02HB013_2 = split_by_param_1_2(_02HB013)
_02HB029_1, _02HB029_2 = split_by_param_1_2(_02HB029)



In [6]:
_02HB018_1_ts = wateroffice_to_timeseries(_02HB018_1).rename(columns={'value': 'discharge_02HB018'})
_02HB018_2_ts = wateroffice_to_timeseries(_02HB018_2).rename(columns={'value': 'water_level_02HB018'})
_02HB025_1_ts = wateroffice_to_timeseries(_02HB025_1).rename(columns={'value': 'discharge_02HB025'})
_02HB025_2_ts = wateroffice_to_timeseries(_02HB025_2).rename(columns={'value': 'water_level_02HB025'})
_02HB031_1_ts = wateroffice_to_timeseries(_02HB031_1).rename(columns={'value': 'discharge_02HB031'})
_02HB031_2_ts = wateroffice_to_timeseries(_02HB031_2).rename(columns={'value': 'water_level_02HB031'})
_02HB001_1_ts = wateroffice_to_timeseries(_02HB001_1).rename(columns={'value': 'discharge_02HB001'})
_02HB001_2_ts = wateroffice_to_timeseries(_02HB001_2).rename(columns={'value': 'water_level_02HB001'})
_02HB013_1_ts = wateroffice_to_timeseries(_02HB013_1).rename(columns={'value': 'discharge_02HB013'})
_02HB013_2_ts = wateroffice_to_timeseries(_02HB013_2).rename(columns={'value': 'water_level_02HB013'})
_02HB029_1_ts = wateroffice_to_timeseries(_02HB029_1).rename(columns={'value': 'discharge_02HB029'})
_02HB029_2_ts = wateroffice_to_timeseries(_02HB029_2).rename(columns={'value': 'water_level_02HB029'})

In [7]:
def merge_param_1_2(df_left : pd.DataFrame, df_right : pd.DataFrame):
    return pd.merge(df_left, df_right, on='timestamp', how='inner')


def fill_na(df: pd.DataFrame):
    # if whole row is NA, drop it, otherwise fill it with ffill and bfill
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    return df


In [8]:
merged_02HB018 = merge_param_1_2(_02HB018_1_ts, _02HB018_2_ts)
merged_02HB025 = merge_param_1_2(_02HB025_1_ts, _02HB025_2_ts)
merged_02HB031 = merge_param_1_2(_02HB031_1_ts, _02HB031_2_ts)
merged_02HB001 = merge_param_1_2(_02HB001_1_ts, _02HB001_2_ts)
merged_02HB013 = merge_param_1_2(_02HB013_1_ts, _02HB013_2_ts)
merged_02HB029 = merge_param_1_2(_02HB029_1_ts, _02HB029_2_ts)

In [9]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['timestamp'],
                                            how='outer'), [merged_02HB018, merged_02HB025, merged_02HB031, merged_02HB001, merged_02HB013, merged_02HB029])

In [10]:
print(df_merged['timestamp'].diff().dt.days.unique())

[nan  1.]


In [11]:
df_merged

,timestamp,discharge_02HB018,water_level_02HB018,discharge_02HB025,water_level_02HB025,discharge_02HB031,water_level_02HB031,discharge_02HB001,water_level_02HB001,discharge_02HB013,water_level_02HB013,discharge_02HB029,water_level_02HB029
0,2002-01-01,2.50,3.944,3.27,2.829,NaN,NaN,1.20,1.475,0.380,11.205,NaN,NaN
1,2002-01-02,2.47,3.983,3.24,2.857,NaN,NaN,1.19,1.473,0.375,11.204,NaN,NaN
2,2002-01-03,2.50,4.005,3.28,2.911,NaN,NaN,1.20,1.474,0.376,11.204,NaN,NaN
3,2002-01-04,2.53,3.943,3.31,2.744,NaN,NaN,1.20,1.474,0.375,11.204,NaN,NaN
4,2002-01-05,2.48,3.898,3.23,2.655,NaN,NaN,1.19,1.474,0.375,11.204,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8030,2023-12-27,4.39,3.931,NaN,NaN,NaN,NaN,NaN,NaN,0.652,11.228,12.40,4.674
8031,2023-12-28,5.68,4.019,NaN,NaN,NaN,NaN,NaN,NaN,0.692,11.236,14.40,4.728
8032,2023-12-29,4.65,3.951,NaN,NaN,NaN,NaN,NaN,NaN,0.536,11.217,11.80,4.673
8033,2023-12-30,4.37,3.932,NaN,NaN,NaN,NaN,NaN,NaN,0.551,11.219,11.20,4.660


In [12]:
df_merged.set_index('timestamp', inplace=True)

df_merged = fill_na(df_merged)

In [13]:
def add_lag_times(df: pd.DataFrame):
    lag_times = {
        '02HB018': 7,
        '02HB025': 5,
        '02HB031': 9,
        '02HB001': 8,
        '02HB013': 12
    }

    df = df_hourly.copy()

    for column in df_hourly.columns:
        for station, lag in lag_times.items():
            if station in column: 
                df[f'{column}_lagged'] = df[column].shift(lag)

    return df

def interpolate_hourly(df: pd.DataFrame):
    df_hourly = df.resample('1H').interpolate('linear')
    df_hourly = fill_na(df_hourly)
    return df_hourly

df_hourly = interpolate_hourly(df_merged)

df_hourly_lagged = add_lag_times(df_hourly)

df_hourly_lagged = fill_na(df_hourly_lagged)


/tmp/ipykernel_2991/1904398415.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('1H').interpolate('linear')


In [14]:
df_hourly

,discharge_02HB018,water_level_02HB018,discharge_02HB025,water_level_02HB025,discharge_02HB031,water_level_02HB031,discharge_02HB001,water_level_02HB001,discharge_02HB013,water_level_02HB013,discharge_02HB029,water_level_02HB029
timestamp,,,,,,,,,,,,
2002-01-01 00:00:00,2.50000,3.944000,3.27000,2.829000,0.288,2.014,1.200000,1.475000,0.380000,11.205000,12.100000,4.563000
2002-01-01 01:00:00,2.49875,3.945625,3.26875,2.830167,0.288,2.014,1.199583,1.474917,0.379792,11.204958,12.100000,4.563000
2002-01-01 02:00:00,2.49750,3.947250,3.26750,2.831333,0.288,2.014,1.199167,1.474833,0.379583,11.204917,12.100000,4.563000
2002-01-01 03:00:00,2.49625,3.948875,3.26625,2.832500,0.288,2.014,1.198750,1.474750,0.379375,11.204875,12.100000,4.563000
2002-01-01 04:00:00,2.49500,3.950500,3.26500,2.833667,0.288,2.014,1.198333,1.474667,0.379167,11.204833,12.100000,4.563000
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30 20:00:00,3.84500,3.894500,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.461833,11.205667,8.958333,4.603333
2023-12-30 21:00:00,3.81875,3.892625,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.457375,11.205000,8.846250,4.600500
2023-12-30 22:00:00,3.79250,3.890750,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.452917,11.204333,8.734167,4.597667


In [15]:
df_hourly_lagged

,discharge_02HB018,water_level_02HB018,discharge_02HB025,water_level_02HB025,discharge_02HB031,water_level_02HB031,discharge_02HB001,water_level_02HB001,discharge_02HB013,water_level_02HB013,...,discharge_02HB018_lagged,water_level_02HB018_lagged,discharge_02HB025_lagged,water_level_02HB025_lagged,discharge_02HB031_lagged,water_level_02HB031_lagged,discharge_02HB001_lagged,water_level_02HB001_lagged,discharge_02HB013_lagged,water_level_02HB013_lagged
timestamp,,,,,,,,,,,,,,,,,,,,,
2002-01-01 00:00:00,2.50000,3.944000,3.27000,2.829000,0.288,2.014,1.200000,1.475000,0.380000,11.205000,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 01:00:00,2.49875,3.945625,3.26875,2.830167,0.288,2.014,1.199583,1.474917,0.379792,11.204958,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 02:00:00,2.49750,3.947250,3.26750,2.831333,0.288,2.014,1.199167,1.474833,0.379583,11.204917,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 03:00:00,2.49625,3.948875,3.26625,2.832500,0.288,2.014,1.198750,1.474750,0.379375,11.204875,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 04:00:00,2.49500,3.950500,3.26500,2.833667,0.288,2.014,1.198333,1.474667,0.379167,11.204833,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30 20:00:00,3.84500,3.894500,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.461833,11.205667,...,4.02875,3.907625,12.30,2.843,0.303,1.969,3.13,1.521,0.515333,11.213667
2023-12-30 21:00:00,3.81875,3.892625,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.457375,11.205000,...,4.00250,3.905750,12.30,2.843,0.303,1.969,3.13,1.521,0.510875,11.213000
2023-12-30 22:00:00,3.79250,3.890750,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.452917,11.204333,...,3.97625,3.903875,12.30,2.843,0.303,1.969,3.13,1.521,0.506417,11.212333


In [16]:
assert(len(df_merged.index.to_series().diff().dt.days.unique()) == 2)
assert(len(df_hourly.index.to_series().diff().dt.days.unique()) == 2)

In [17]:
df_hourly.head(2)


,discharge_02HB018,water_level_02HB018,discharge_02HB025,water_level_02HB025,discharge_02HB031,water_level_02HB031,discharge_02HB001,water_level_02HB001,discharge_02HB013,water_level_02HB013,discharge_02HB029,water_level_02HB029
timestamp,,,,,,,,,,,,
2002-01-01 00:00:00,2.50000,3.944000,3.27000,2.829000,0.288,2.014,1.200000,1.475000,0.380000,11.205000,12.1,4.563
2002-01-01 01:00:00,2.49875,3.945625,3.26875,2.830167,0.288,2.014,1.199583,1.474917,0.379792,11.204958,12.1,4.563


In [18]:
df_hourly_lagged

,discharge_02HB018,water_level_02HB018,discharge_02HB025,water_level_02HB025,discharge_02HB031,water_level_02HB031,discharge_02HB001,water_level_02HB001,discharge_02HB013,water_level_02HB013,...,discharge_02HB018_lagged,water_level_02HB018_lagged,discharge_02HB025_lagged,water_level_02HB025_lagged,discharge_02HB031_lagged,water_level_02HB031_lagged,discharge_02HB001_lagged,water_level_02HB001_lagged,discharge_02HB013_lagged,water_level_02HB013_lagged
timestamp,,,,,,,,,,,,,,,,,,,,,
2002-01-01 00:00:00,2.50000,3.944000,3.27000,2.829000,0.288,2.014,1.200000,1.475000,0.380000,11.205000,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 01:00:00,2.49875,3.945625,3.26875,2.830167,0.288,2.014,1.199583,1.474917,0.379792,11.204958,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 02:00:00,2.49750,3.947250,3.26750,2.831333,0.288,2.014,1.199167,1.474833,0.379583,11.204917,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 03:00:00,2.49625,3.948875,3.26625,2.832500,0.288,2.014,1.198750,1.474750,0.379375,11.204875,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
2002-01-01 04:00:00,2.49500,3.950500,3.26500,2.833667,0.288,2.014,1.198333,1.474667,0.379167,11.204833,...,2.50000,3.944000,3.27,2.829,0.288,2.014,1.20,1.475,0.380000,11.205000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30 20:00:00,3.84500,3.894500,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.461833,11.205667,...,4.02875,3.907625,12.30,2.843,0.303,1.969,3.13,1.521,0.515333,11.213667
2023-12-30 21:00:00,3.81875,3.892625,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.457375,11.205000,...,4.00250,3.905750,12.30,2.843,0.303,1.969,3.13,1.521,0.510875,11.213000
2023-12-30 22:00:00,3.79250,3.890750,12.30000,2.843000,0.303,1.969,3.130000,1.521000,0.452917,11.204333,...,3.97625,3.903875,12.30,2.843,0.303,1.969,3.13,1.521,0.506417,11.212333


In [19]:
df_merged

,discharge_02HB018,water_level_02HB018,discharge_02HB025,water_level_02HB025,discharge_02HB031,water_level_02HB031,discharge_02HB001,water_level_02HB001,discharge_02HB013,water_level_02HB013,discharge_02HB029,water_level_02HB029
timestamp,,,,,,,,,,,,
2002-01-01,2.50,3.944,3.27,2.829,0.288,2.014,1.20,1.475,0.380,11.205,12.10,4.563
2002-01-02,2.47,3.983,3.24,2.857,0.288,2.014,1.19,1.473,0.375,11.204,12.10,4.563
2002-01-03,2.50,4.005,3.28,2.911,0.288,2.014,1.20,1.474,0.376,11.204,12.10,4.563
2002-01-04,2.53,3.943,3.31,2.744,0.288,2.014,1.20,1.474,0.375,11.204,12.10,4.563
2002-01-05,2.48,3.898,3.23,2.655,0.288,2.014,1.19,1.474,0.375,11.204,12.10,4.563
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,4.39,3.931,12.30,2.843,0.303,1.969,3.13,1.521,0.652,11.228,12.40,4.674
2023-12-28,5.68,4.019,12.30,2.843,0.303,1.969,3.13,1.521,0.692,11.236,14.40,4.728
2023-12-29,4.65,3.951,12.30,2.843,0.303,1.969,3.13,1.521,0.536,11.217,11.80,4.673


In [20]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, BatchNormalization, Bidirectional, LayerNormalization, ConvLSTM1D, Flatten, Dropout

def make_lstm(shape):

    #_LSTM = Sequential()
    #_LSTM.add(LSTM(100, return_sequences=True, input_shape=input_shape, dropout=0.2, recurrent_dropout=0.2))
    #_LSTM.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    #_LSTM.add(Dense(1))
    _LSTM = Sequential()
    _LSTM.add(Bidirectional(LSTM(100, return_sequences=True, input_shape=(1,shape), dropout=0.2, recurrent_dropout=0.2)))
    _LSTM.add(BatchNormalization())
    _LSTM.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2)))
    _LSTM.add(BatchNormalization())
    _LSTM.add(Dense(1))

    return _LSTM

def make_gru(shape):

    _GRU = Sequential()
    _GRU.add(GRU(50, return_sequences=True, input_shape=(1,shape), dropout=0.2))
    _GRU.add(LayerNormalization())
    _GRU.add(GRU(25, dropout=0.2))
    _GRU.add(Dense(1))

    return _GRU

def make_conv_lstm(shape):
    _CONVLSTM = Sequential()
    _CONVLSTM.add(ConvLSTM1D(filters=32, kernel_size=3, return_sequences=True, input_shape=(1,shape,1)))
    _CONVLSTM.add(BatchNormalization())
    #_CNNLSTM.add(ConvLSTM1D(filters=32, kernel_size=3, return_sequences=False))
    #_CNNLSTM.add(BatchNormalization())
    _CONVLSTM.add(Flatten())
    _CONVLSTM.add(Dense(50, activation='relu'))
    _CONVLSTM.add(Dropout(0.25))
    _CONVLSTM.add(Dense(1))

    return _CONVLSTM

In [21]:
def prepare_dataset(df: pd.DataFrame):

    scaler = MinMaxScaler(feature_range=(0, 1))
    df_scaled = scaler.fit_transform(df)

    def create_dataset(data :np.array, look_back=1):
        """
        set y as the next value in the sequence at the last index (water_level_02HB029)
        """
        X, Y = [], []
        for i in range(len(data) - look_back):
            a = data[i:(i + look_back), :]
            X.append(a)
            Y.append(data[i + look_back, -1])
        return np.array(X), np.array(Y) 
    
    look_back = 1
    X, Y = create_dataset(df_scaled, look_back)

    test_size = int(len(X) * 0.20)
    train_size = len(X) - test_size
    val_size = int(train_size * 0.20)

    X_train, X_val = X[:train_size-val_size], X[train_size-val_size:train_size]
    Y_train, Y_val = Y[:train_size-val_size], Y[train_size-val_size:train_size]
    X_test, Y_test = X[train_size:], Y[train_size:]

    return X_train, Y_train, X_val, Y_val, X_test, Y_test, scaler


In [22]:
# # df_hourly_lagged as a string to file

# with open('df_hourly.txt', 'w') as f:
#     f.write(df_hourly_lagged.to_string())


# # read in df_hourly_lagged from file
# df = pd.read_csv('df_hourly_lagged.txt', sep='\s+', engine='python')


In [23]:

# from string to dataframea
df_hourly_lagged = pd.read_csv(pd.compat.StringIO(s), sep='\s+')

AttributeError: module 'pandas.compat' has no attribute 'StringIO'

In [ ]:






datasets = {
    'merged': df_merged,
    'hourly': df_hourly,
    'lagged': df_hourly_lagged
}

models = {
    'LSTM': make_lstm,
    'GRU': make_gru,
    'ConvLSTM': make_conv_lstm
}

results = {}

for dataset_name, df in datasets.items():
    X_train, Y_train, X_val, Y_val, X_test, Y_test, scaler = prepare_dataset(df)
    for model_name, model_builder in models.items():
        
        model = model_builder(df.shape[1])

        model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
        history = model.fit(X_train, Y_train, epochs=100, batch_size=256, verbose=2,
                     validation_data=(X_val, Y_val))
        
        predictions = model.predict(X_test)
        predictions = scaler.inverse_transform(
            np.hstack((np.zeros((predictions.shape[0], df.shape[1]-1)), predictions)))

        Y_test = scaler.inverse_transform(
            np.hstack((np.zeros((Y_test.shape[0], df.shape[1]-1)), Y_test.reshape(-1, 1))))[:, -1]

        predictions = predictions[:, -1]

        rmse = math.sqrt(mean_squared_error(Y_test, predictions))
        mae = np.mean(np.abs(predictions - Y_test))
        
        results[(dataset_name, model_name)] = {
            'history': history.history,
            'rmse': rmse,
            'mae': mae,
            'predictions': predictions,
            'Y_test': Y_test, 
            'name': f'{dataset_name}_{model_name}'
        }


In [ ]:
for key, value in results.items():
    print(key, value['rmse'], value['mae'])

In [ ]:
def plot_model_losses(results, dataset_name, model_name):
    history = results[(dataset_name, model_name)]['history']
    plt.figure(figsize=(10, 5))
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name} on {dataset_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    #plt.text(1, 1, f'{results[(dataset_name, model_name)]["name"]}\nMAE: {results[(dataset_name, model_name)]["mae"]}')
    # plt.savefig(f'plots/{dataset_name}_{model_name}_loss.png')
    plt.show()

for dataset_name, model_name in results.keys():
    plot_model_losses(results, dataset_name, model_name) 
